# Semantic Kernel Multi-Agent Orchestration

Welcome to the multi-agent orchestration tutorial! This notebook explores different patterns for coordinating multiple AI agents working together.

## Prerequisites

Before starting this tutorial, make sure you've completed:
- **Semantic Kernel Basics** notebook
- **Advanced Agent Features** notebook

You should be familiar with:
- Agent creation and configuration
- Plugins and function calling
- Streaming responses and intermediate steps
- Structured outputs

## What You'll Learn

- **Group Chat Orchestration**: Round-robin and custom managers
- **Concurrent Processing**: Parallel agent execution
- **Sequential Workflows**: Pipeline-based processing
- **Handoff Systems**: Specialized agent routing
- **Human-in-the-Loop**: Interactive agent workflows

## Resources
- [Agent Framework Guide](https://learn.microsoft.com/en-us/semantic-kernel/frameworks/agent/)

Let's explore multi-agent orchestration patterns!

## Setup and Imports

Let's start by importing all the necessary modules for orchestration.

In [ ]:
import asyncio
import json
from typing import Annotated
import os
from dotenv import load_dotenv
from pydantic import BaseModel

# Core Semantic Kernel imports
from semantic_kernel import Kernel
from semantic_kernel.agents import Agent, ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, OpenAIChatCompletion
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings, AzureChatPromptExecutionSettings
from semantic_kernel.functions import kernel_function, KernelArguments
from semantic_kernel.contents import ChatMessageContent, TextContent, StreamingChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent

# Orchestration imports
from semantic_kernel.agents import (
    GroupChatOrchestration, 
    RoundRobinGroupChatManager,
    ConcurrentOrchestration,
    SequentialOrchestration,
    HandoffOrchestration,
    OrchestrationHandoffs
)
from semantic_kernel.agents.runtime import InProcessRuntime
from semantic_kernel.agents.orchestration.tools import structured_outputs_transform
from semantic_kernel.agents.orchestration.group_chat import BooleanResult, GroupChatManager, MessageResult, StringResult
from semantic_kernel.connectors.ai.prompt_execution_settings import PromptExecutionSettings
from semantic_kernel.prompt_template import KernelPromptTemplate, PromptTemplateConfig
from typing_extensions import override

# Additional imports for LLM-based group chat manager
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase

# Load environment variables
load_dotenv()

print("✅ Orchestration imports loaded successfully!")

In [2]:
# Configure the main chat completion service
chat_completion = AzureChatCompletion(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
)

print("✅ Chat completion service configured!")

✅ Chat completion service configured!


## 1. Group Chat Orchestration

Group chat orchestration allows multiple agents to participate in a conversation, taking turns based on a management strategy.

### 1.1 Round Robin Group Chat

Round robin orchestration ensures agents take turns in a specific order.

In [18]:
def create_content_team() -> list[Agent]:
    """Create a team of content creation agents."""
    
    writer = ChatCompletionAgent(
        name="ContentWriter",
        description="A creative content writer.",
        instructions="You create engaging content. Focus on creativity and readability.",
        service=chat_completion,
    )
    
    editor = ChatCompletionAgent(
        name="ContentEditor",
        description="A content editor and reviewer.",
        instructions="You review and improve content. Focus on clarity, grammar, and flow.",
        service=chat_completion,
    )
    
    return [writer, editor]

def agent_response_callback(message: ChatMessageContent) -> None:
    """Display agent responses with truncation for cleaner logs."""
    print(f"**{message.name}**")
    
    # Truncate long responses for cleaner output
    content = message.content
    if len(content) > 500:
        content = content[:500] + "... [truncated]"
    
    print(content)
    print("-" * 50)

# Create the group chat
content_agents = create_content_team()
group_chat = GroupChatOrchestration(
    members=content_agents,
    manager=RoundRobinGroupChatManager(max_rounds=3),
    agent_response_callback=agent_response_callback,
)

print("✅ Group chat orchestration created!")

✅ Group chat orchestration created!


In [4]:
# Test group chat orchestration
runtime = InProcessRuntime()
runtime.start()

print("🎯 Task: Create content for a sustainable living blog post")
print("=" * 60)

try:
    orchestration_result = await group_chat.invoke(
        task="Create a compelling blog post about sustainable living tips for busy professionals.",
        runtime=runtime,
    )
    
    final_result = await orchestration_result.get()
    print(f"\n**Final Result**")
    print(final_result)
    
finally:
    await runtime.stop_when_idle()

🎯 Task: Create content for a sustainable living blog post
**ContentWriter**
Sustainable Living for Busy Professionals: Simple Tips You Can Start Today

Introduction  
Juggling work deadlines, family commitments, and a social life can leave little room for eco-friendly choices. Yet, even small sustainable changes can make a big difference—both for the planet and your own well-being. Here’s how busy professionals can weave green habits into a packed schedule.

1. Plan Your Meals, Waste Less  
  • Weekly Meal Prep: Spend one hour on Sunday chopping vegetables, cooking grains, and portioning meals. Homemade lunches reduce takeout packaging and save money.  
  • Smart Shopping Lists: Check your pantry before making a list. Buy only what you need to avoid food waste.  
  • Zero-Waste Snacking: Invest in reusable snack bags or glass jars. Fill them with nuts, fruit, or trail mix before you dash out the door.

2. Transform Your Commute  
  • Carpool or Ride-Share: Coordinate with a coworker or

## 2. Concurrent Orchestration

Concurrent orchestration allows multiple agents to work on the same task simultaneously.

In [5]:
def create_research_team() -> list[Agent]:
    """Create a team of research specialists."""
    
    tech_researcher = ChatCompletionAgent(
        name="TechResearcher",
        instructions="You are a technology research expert. Analyze questions from a technical perspective.",
        service=chat_completion,
    )
    
    market_researcher = ChatCompletionAgent(
        name="MarketResearcher",
        instructions="You are a market research expert. Analyze questions from a business and market perspective.",
        service=chat_completion,
    )
    
    return [tech_researcher, market_researcher]

# Create concurrent orchestration
research_agents = create_research_team()
concurrent_orchestration = ConcurrentOrchestration(members=research_agents)

print("✅ Concurrent orchestration created!")

✅ Concurrent orchestration created!


In [6]:
# Test concurrent orchestration
runtime = InProcessRuntime()
runtime.start()

print("🎯 Research Topic: Electric Vehicle Market")
print("=" * 50)

try:
    orchestration_result = await concurrent_orchestration.invoke(
        task="Analyze the current state and future prospects of the electric vehicle market.",
        runtime=runtime,
    )
    
    results = await orchestration_result.get(timeout=60)
    
    for result in results:
        print(f"**{result.name}**")
        print(result.content)
        print("-" * 50)
        
finally:
    await runtime.stop_when_idle()

🎯 Research Topic: Electric Vehicle Market
**TechResearcher**
Below is an overview of where the global electric-vehicle (EV) market stands today, the main drivers and barriers it faces, and the key trends likely to shape its evolution over the next decade.  

1. Market Size and Growth  
• Global Sales and Penetration  
  – In 2023, roughly 14–16% of new-vehicle sales worldwide were battery electric vehicles (BEVs), up from about 4% in 2020. Plug-in hybrids (PHEVs) added another 4–5%.  
  – China leads with more than half of global EV sales, followed by Europe (~30%) and North America (~15%).  
• Manufacturers and Models  
  – Tesla remains the top BEV seller by volume, but legacy automakers (Volkswagen Group, Stellantis, Hyundai-Kia, GM, Ford) are rapidly ramping dedicated EV platforms.  
  – New entrants (NIO, XPeng, Rivian, BYD) are expanding beyond home markets.  

2. Key Enablers Today  
• Battery Cost Declines  
  – Average pack prices fell from ~$1,100/kWh in 2010 to ~$135/kWh in 

### 2.1 Concurrent Orchestration with Structured Outputs

You can combine concurrent orchestration with structured outputs for consistent results.

In [7]:
# Define structured output for research
class ResearchInsight(BaseModel):
    """Research insight with key findings."""
    key_finding: str
    supporting_evidence: str
    implications: str

class ResearchPerspectives(BaseModel):
    """Combined research perspectives."""
    technology: ResearchInsight
    market: ResearchInsight

# Create structured concurrent orchestration
structured_concurrent = ConcurrentOrchestration[str, ResearchPerspectives](
    members=research_agents,
    output_transform=structured_outputs_transform(ResearchPerspectives, chat_completion),
)

print("✅ Structured concurrent orchestration created!")

✅ Structured concurrent orchestration created!


In [8]:
# Test structured concurrent orchestration
runtime = InProcessRuntime()
runtime.start()

print("🎯 Structured Research: AI in Healthcare")
print("=" * 50)

try:
    orchestration_result = await structured_concurrent.invoke(
        task="Research AI applications in healthcare. Provide key finding, evidence, and implications.",
        runtime=runtime,
    )
    
    result = await orchestration_result.get(timeout=60)
    
    if isinstance(result, ResearchPerspectives):
        print("📊 Structured Research Results:")
        print(result.model_dump_json(indent=2))
    else:
        print(f"⚠️ Unexpected result type: {type(result)}")
        
finally:
    await runtime.stop_when_idle()

🎯 Structured Research: AI in Healthcare
📊 Structured Research Results:
{
  "technology": {
    "key_finding": "AI technologies across imaging, predictive analytics, genomics, drug discovery, NLP and remote monitoring have demonstrated superior accuracy and throughput compared with traditional methods, enabling earlier disease detection, personalized treatment and streamlined clinical operations.",
    "supporting_evidence": "– Imaging: CNN models achieved AUC 0.94 on chest X-ray pathologies and reduced recall rates by 5% in mammography while maintaining >90% sensitivity (Nature Medicine 2020; JAMA 2022).\n– Predictive analytics: EHR-based ML models deliver AUCs of 0.75–0.85 for deterioration alerts and 0.82 for 30-day readmission prediction vs. 0.71 for standard scores (BMJ 2019; Epic Deterioration Index).  \n– Genomics: ML predicted immunotherapy response in melanoma with 78% accuracy vs. 55% for PD-L1 assays (Lancet Oncology 2021); DeepVariant improves variant calls by ~50%.  \n– Dru

## 3. Sequential Orchestration

Sequential orchestration passes the output of one agent as input to the next, creating a pipeline.

In [9]:
def create_product_pipeline() -> list[Agent]:
    """Create a product development pipeline."""
    
    researcher = ChatCompletionAgent(
        name="ProductResearcher",
        instructions="You analyze product ideas and identify key features, target audience, and market opportunities.",
        service=chat_completion,
    )
    
    designer = ChatCompletionAgent(
        name="ProductDesigner",
        instructions="You create product concepts and specifications based on research insights.",
        service=chat_completion,
    )
    
    marketer = ChatCompletionAgent(
        name="ProductMarketer",
        instructions="You create marketing strategies and compelling messaging based on product designs.",
        service=chat_completion,
    )
    
    return [researcher, designer, marketer]

# Create sequential orchestration
product_pipeline = create_product_pipeline()
sequential_orchestration = SequentialOrchestration(
    members=product_pipeline,
    agent_response_callback=agent_response_callback,
)

print("✅ Sequential orchestration created!")

✅ Sequential orchestration created!


In [10]:
# Test sequential orchestration
runtime = InProcessRuntime()
runtime.start()

print("🎯 Product Development Pipeline: Smart Home Security Device")
print("=" * 60)

try:
    orchestration_result = await sequential_orchestration.invoke(
        task="Develop a smart home security device that uses AI for threat detection.",
        runtime=runtime,
    )
    
    final_result = await orchestration_result.get(timeout=120)
    print(f"\n**Final Marketing Strategy**")
    print(final_result)
    
finally:
    await runtime.stop_when_idle()

🎯 Product Development Pipeline: Smart Home Security Device
**ProductResearcher**
Product Concept Summary  
A next-generation smart home security device that leverages on-device and cloud-assisted AI to detect, classify and respond to threats in real time—without overwhelming homeowners with false alarms. It combines advanced computer vision, audio analysis and sensor fusion in a compact, modular design that integrates with existing smart-home ecosystems.

Key Features  
1. AI-Driven Threat Detection  
  • Computer vision models trained to recognize people vs. animals, package deliveries, loitering, forced entry, weapons and aggressive behavior  
  • Acoustic analysis for glass-break, screaming, alarms or unusual sounds  
  • Multi-sensor fusion (PIR motion, radar, audio, vibration) to reduce false positives  

2. Edge-Cloud Hybrid Processing  
  • On-device inference for privacy and low latency; critical alerts processed locally  
  • Optional cloud processing for advanced analytics (e

### 3.1 Streaming and intermediate results with Orchestration

Lets looks at how you can use streaming and observe intermediate results with the sequential pattern. This will work for other orchestration patterns too.

In [22]:
# Global variable to track streaming state
is_new_message = True

def streaming_agent_response_callback(message: StreamingChatMessageContent, is_final: bool) -> None:
    """Handle streaming responses from agents."""
    global is_new_message
    
    if is_new_message:
        print(f"\n**{message.name}**")
        is_new_message = False
        
    print(message.content, end="", flush=True)
    
    if is_final:
        print("\n" + "-" * 50)
        is_new_message = True

def agent_response_callback(message: ChatMessageContent) -> None:
    """Handle support agent responses."""
    
    # Show function calls
    for item in message.items:
        if isinstance(item, FunctionCallContent):
            print(f"  📞 Calling: {item.name}({item.arguments})")
        elif isinstance(item, FunctionResultContent):
            print(f"  📋 Result: {item.result}")


class AnalysisPlugin:
    """Plugin that provides analysis capabilities."""
    
    @kernel_function(description="Analyze a topic and provide insights.")
    def analyze_topic(self, topic: Annotated[str, "The topic to analyze"]) -> str:
        # Simulate analysis processing
        return f"Analysis of '{topic}': This is a complex topic that requires careful consideration of multiple factors including market trends, user needs, and technical feasibility."
    
    @kernel_function(description="Generate recommendations based on analysis.")
    def generate_recommendations(self, analysis: Annotated[str, "The analysis to base recommendations on"]) -> str:
        return "Recommendations: 1) Conduct further research, 2) Develop a prototype, 3) Test with users, 4) Iterate based on feedback."

analysis_agent = ChatCompletionAgent(
    service=chat_completion,
    name="AnalysisExpert",
    instructions="You are an expert analyst. Use your tools to provide thorough analysis and recommendations.",
    plugins=[AnalysisPlugin()]
)

summarizer_agent = ChatCompletionAgent(
    service=chat_completion,
    name="Summarizer",
    instructions="You summarize results in no more than 100 words.",
)

# Create streaming sequential orchestration
streaming_sequential = SequentialOrchestration(
    members=[analysis_agent, summarizer_agent], ## Resusing reasoning agent for demonstration purposes.
    streaming_agent_response_callback=streaming_agent_response_callback,
    agent_response_callback=agent_response_callback
)

print("✅ Streaming sequential orchestration created!")


✅ Streaming sequential orchestration created!


In [23]:
# Test streaming sequential orchestration
runtime = InProcessRuntime()
runtime.start()

print("🎯 Streaming Product Pipeline: Fitness Wearable")
print("=" * 60)

try:
    orchestration_result = await streaming_sequential.invoke(
        task="Create a fitness wearable that tracks mental wellness alongside physical health.",
        runtime=runtime,
    )
    
    final_result = await orchestration_result.get(timeout=120)
    print(f"\n\n**Final Pipeline Result**")
    print(final_result)
    
finally:
    await runtime.stop_when_idle()

🎯 Streaming Product Pipeline: Fitness Wearable
  📞 Calling: AnalysisPlugin-analyze_topic({"topic":"fitness wearable that tracks mental wellness alongside physical health"})

**AnalysisExpert**

--------------------------------------------------
  📋 Result: Analysis of 'fitness wearable that tracks mental wellness alongside physical health': This is a complex topic that requires careful consideration of multiple factors including market trends, user needs, and technical feasibility.

**AnalysisExpert**

--------------------------------------------------

**AnalysisExpert**
  📞 Calling: AnalysisPlugin-generate_recommendations({"analysis":"Analysis of 'fitness wearable that tracks mental wellness alongside physical health':\n\n1. Market Trends and User Needs:\n   - Rising Demand for Holistic Health Solutions: Consumers increasingly seek devices that offer both physical and mental health insights in one package.\n   - Mental Wellness Awareness Growth: Post-pandemic, mental health tracking 

## 4. LLM-Based Group Chat Manager

For more sophisticated group interactions, you can create LLM-based group chat managers that use AI to intelligently decide who speaks next based on conversation context and participant expertise.

In [19]:
class LLMBasedGroupChatManager(GroupChatManager):
    """LLM-based group chat manager that uses AI to decide who speaks next.
    
    This manager uses the LLM to intelligently select the next speaker based on
    conversation context and participant expertise. Key features:
    
    - Uses AI to determine when the discussion should end
    - Selects speakers based on conversation context and agent descriptions
    - Provides reasoning for each selection and termination decision
    - Generates intelligent summaries of the discussion
    """

    service: ChatCompletionClientBase
    topic: str

    termination_prompt: str = (
        "You are mediator that guides a discussion on the topic of '{{$topic}}'. "
        "You need to determine if the discussion has reached a conclusion. "
        "If you would like to end the discussion, please respond with True. Otherwise, respond with False."
    )

    selection_prompt: str = (
        "You are mediator that guides a discussion on the topic of '{{$topic}}'. "
        "You need to select the next participant to speak. "
        "Here are the names and descriptions of the participants: "
        "{{$participants}}\n"
        "Please respond with only the name of the participant you would like to select."
    )

    result_filter_prompt: str = (
        "You are mediator that guides a discussion on the topic of '{{$topic}}'. "
        "You have just concluded the discussion. "
        "Please summarize the discussion and provide a closing statement."
    )

    def __init__(self, topic: str, service: ChatCompletionClientBase, **kwargs) -> None:
        """Initialize the group chat manager."""
        super().__init__(topic=topic, service=service, **kwargs)

    async def _render_prompt(self, prompt: str, arguments: KernelArguments) -> str:
        """Helper to render a prompt with arguments."""
        prompt_template_config = PromptTemplateConfig(template=prompt)
        prompt_template = KernelPromptTemplate(prompt_template_config=prompt_template_config)
        return await prompt_template.render(Kernel(), arguments=arguments)

    @override
    async def should_request_user_input(self, chat_history: ChatHistory) -> BooleanResult:
        """Provide concrete implementation for determining if user input is needed.

        The manager will check if input from human is needed after each agent message.
        """
        return BooleanResult(
            result=False,
            reason="This group chat manager does not require user input.",
        )

    @override
    async def should_terminate(self, chat_history: ChatHistory) -> BooleanResult:
        """Provide concrete implementation for determining if the discussion should end.

        The manager will check if the conversation should be terminated after each agent message
        or human input (if applicable).
        """
        should_terminate = await super().should_terminate(chat_history)
        if should_terminate.result:
            return should_terminate

        chat_history.messages.insert(
            0,
            ChatMessageContent(
                role=AuthorRole.SYSTEM,
                content=await self._render_prompt(
                    self.termination_prompt,
                    KernelArguments(topic=self.topic),
                ),
            ),
        )
        chat_history.add_message(
            ChatMessageContent(role=AuthorRole.USER, content="Determine if the discussion should end."),
        )

        response = await self.service.get_chat_message_content(
            chat_history,
            settings=PromptExecutionSettings(response_format=BooleanResult),
        )

        termination_with_reason = BooleanResult.model_validate_json(response.content)

        print("*********************")
        print(f"🤖 LLM Decision - Should terminate: {termination_with_reason.result}")
        print(f"📝 Reason: {termination_with_reason.reason}")
        print("*********************")

        return termination_with_reason

    @override
    async def select_next_agent(
        self,
        chat_history: ChatHistory,
        participant_descriptions: dict[str, str],
    ) -> StringResult:
        """Provide concrete implementation for selecting the next agent to speak.

        The manager will select the next agent to speak after each agent message
        or human input (if applicable) if the conversation is not terminated.
        """
        chat_history.messages.insert(
            0,
            ChatMessageContent(
                role=AuthorRole.SYSTEM,
                content=await self._render_prompt(
                    self.selection_prompt,
                    KernelArguments(
                        topic=self.topic,
                        participants="\n".join([f"{k}: {v}" for k, v in participant_descriptions.items()]),
                    ),
                ),
            ),
        )
        chat_history.add_message(
            ChatMessageContent(role=AuthorRole.USER, content="Now select the next participant to speak."),
        )

        response = await self.service.get_chat_message_content(
            chat_history,
            settings=PromptExecutionSettings(response_format=StringResult),
        )

        participant_name_with_reason = StringResult.model_validate_json(response.content)

        print("*********************")
        print(f"🤖 LLM Selection - Next participant: {participant_name_with_reason.result}")
        print(f"📝 Reason: {participant_name_with_reason.reason}")
        print("*********************")

        if participant_name_with_reason.result in participant_descriptions:
            return participant_name_with_reason

        raise RuntimeError(f"Unknown participant selected: {response.content}.")

    @override
    async def filter_results(
        self,
        chat_history: ChatHistory,
    ) -> MessageResult:
        """Provide concrete implementation for filtering the results of the discussion.

        The manager will filter the results of the conversation after the conversation is terminated.
        """
        if not chat_history.messages:
            raise RuntimeError("No messages in the chat history.")

        chat_history.messages.insert(
            0,
            ChatMessageContent(
                role=AuthorRole.SYSTEM,
                content=await self._render_prompt(
                    self.result_filter_prompt,
                    KernelArguments(topic=self.topic),
                ),
            ),
        )
        chat_history.add_message(
            ChatMessageContent(role=AuthorRole.USER, content="Please summarize the discussion."),
        )

        response = await self.service.get_chat_message_content(
            chat_history,
            settings=PromptExecutionSettings(response_format=StringResult),
        )
        string_with_reason = StringResult.model_validate_json(response.content)

        return MessageResult(
            result=ChatMessageContent(role=AuthorRole.ASSISTANT, content=string_with_reason.result),
            reason=string_with_reason.reason,
        )

print("✅ LLM-based Group Chat Manager created!")

✅ LLM-based Group Chat Manager created!


In [20]:
# Create debate agents with rich descriptions for LLM selection
def create_debate_agents() -> list[Agent]:
    """Create agents for a debate scenario."""
    
    optimist = ChatCompletionAgent(
        name="TechOptimist",
        description="A technology optimist who believes in AI's positive potential and focuses on benefits, opportunities, and solutions.",
        instructions="You believe AI will greatly benefit humanity. Present positive arguments with specific examples and address concerns constructively.",
        service=chat_completion,
    )
    
    skeptic = ChatCompletionAgent(
        name="TechSkeptic", 
        description="A technology skeptic who is concerned about AI risks and challenges, focusing on potential problems and ethical concerns.",
        instructions="You are concerned about AI risks and challenges. Present thoughtful counterarguments and highlight potential dangers while being respectful.",
        service=chat_completion,
    )
    
    return [optimist, skeptic]

# Create LLM-based group chat
debate_agents = create_debate_agents()
llm_group_chat = GroupChatOrchestration(
    members=debate_agents,
    manager=LLMBasedGroupChatManager(
        topic="The Future of AI in Society",
        service=chat_completion,
        max_rounds=8,
    ),
    agent_response_callback=agent_response_callback,
)

print("✅ LLM-based group chat for debate created!")

✅ LLM-based group chat for debate created!


In [21]:
# Test LLM-based group chat
runtime = InProcessRuntime()
runtime.start()

print("🎯 AI Debate: The Future of AI in Society")
print("🤖 LLM Manager will intelligently decide who speaks next based on context")
print("=" * 60)

try:
    orchestration_result = await llm_group_chat.invoke(
        task="Discuss the benefits and risks of AI integration in society. Consider both opportunities and challenges.",
        runtime=runtime,
    )
    
    final_result = await orchestration_result.get()
    print(f"\n**LLM-Mediated Debate Summary**")
    print(final_result)
    
finally:
    await runtime.stop_when_idle()

🎯 AI Debate: The Future of AI in Society
🤖 LLM Manager will intelligently decide who speaks next based on context
*********************
🤖 LLM Decision - Should terminate: False
📝 Reason: The discussion has not yet covered the benefits and risks of AI integration; further exploration is needed.
*********************
*********************
🤖 LLM Selection - Next participant: TechOptimist
📝 Reason: Starting with the optimistic view will highlight the potential benefits and opportunities of AI integration in society.
*********************
**TechOptimist**
Below is a balanced overview of how AI can be integrated into society, highlighting both its promise and the challenges we must address.

Benefits and Opportunities  
• Healthcare improvements  
  – Early diagnosis: AI algorithms can spot patterns in medical imaging (e.g., detecting tumors in X-rays or MRIs) faster than the human eye.  
  – Personalized treatment: Machine-learning models analyze genetic and lifestyle data to tailor therapi

## 5. Handoff Orchestration

Handoff orchestration allows agents to transfer conversations to specialized agents based on the context.

In [22]:
# Create specialized support plugins
class TechnicalSupportPlugin:
    """Plugin for technical support tasks."""
    
    @kernel_function(description="Diagnose technical issues.")
    def diagnose_issue(self, issue: Annotated[str, "Description of the technical issue"]) -> str:
        return f"Diagnosis for '{issue}': This appears to be a common connectivity issue. Try restarting your device and checking your network connection."
    
    @kernel_function(description="Provide troubleshooting steps.")
    def troubleshoot(self, diagnosis: Annotated[str, "The diagnosis result"]) -> str:
        return "Troubleshooting steps: 1) Restart device, 2) Check network, 3) Update software, 4) Contact support if issue persists."

class BillingSupportPlugin:
    """Plugin for billing support tasks."""
    
    @kernel_function(description="Check billing information.")
    def check_billing(self, account_id: Annotated[str, "Account ID"]) -> str:
        return f"Billing info for account {account_id}: Current balance: $45.99, Next payment due: 2024-02-15"
    
    @kernel_function(description="Process payment.")
    def process_payment(self, amount: Annotated[str, "Payment amount"]) -> str:
        return f"Payment of {amount} processed successfully. Thank you!"

print("✅ Support plugins created!")

✅ Support plugins created!


In [23]:
# Create specialized support agents
triage_agent = ChatCompletionAgent(
    name="SupportTriage",
    description="A triage agent that routes customers to appropriate specialists.",
    instructions="You help customers by understanding their needs and routing them to the right specialist. Be friendly and efficient.",
    service=chat_completion,
)

tech_support_agent = ChatCompletionAgent(
    name="TechSupport",
    description="A technical support specialist.",
    instructions="You provide technical support and troubleshooting. Use your tools to diagnose and solve problems.",
    service=chat_completion,
    plugins=[TechnicalSupportPlugin()],
)

billing_support_agent = ChatCompletionAgent(
    name="BillingSupport",
    description="A billing support specialist.",
    instructions="You handle billing questions and payment processing. Use your tools to help with account issues.",
    service=chat_completion,
    plugins=[BillingSupportPlugin()],
)

print("✅ Support agents created!")

✅ Support agents created!


In [24]:
# Define handoff relationships
handoffs = (
    OrchestrationHandoffs()
    .add_many(
        source_agent=triage_agent.name,
        target_agents={
            tech_support_agent.name: "Transfer to this agent for technical issues and troubleshooting",
            billing_support_agent.name: "Transfer to this agent for billing, payment, and account issues",
        },
    )
    .add(
        source_agent=tech_support_agent.name,
        target_agent=triage_agent.name,
        description="Transfer back to triage if the issue is not technical",
    )
    .add(
        source_agent=billing_support_agent.name,
        target_agent=triage_agent.name,
        description="Transfer back to triage if the issue is not billing-related",
    )
)

print("✅ Handoff relationships defined!")

✅ Handoff relationships defined!


In [25]:
# Create handoff orchestration
def support_response_callback(message: ChatMessageContent) -> None:
    """Handle support agent responses."""
    print(f"**{message.name}**: {message.content}")
    
    # Show function calls
    for item in message.items:
        if isinstance(item, FunctionCallContent):
            print(f"  📞 Calling: {item.name}({item.arguments})")
        elif isinstance(item, FunctionResultContent):
            print(f"  📋 Result: {item.result}")
    print("-" * 50)

handoff_orchestration = HandoffOrchestration(
    members=[
        triage_agent,
        tech_support_agent,
        billing_support_agent,
    ],
    handoffs=handoffs,
    agent_response_callback=support_response_callback,
)

print("✅ Handoff orchestration created!")

✅ Handoff orchestration created!


In [26]:
# Test handoff orchestration
runtime = InProcessRuntime()
runtime.start()

print("🎯 Customer Support Scenario: Technical Issue")
print("=" * 60)

try:
    orchestration_result = await handoff_orchestration.invoke(
        task="Hi, I'm having trouble connecting to the internet. My device keeps disconnecting.",
        runtime=runtime,
    )
    
    final_result = await orchestration_result.get()
    print(f"\n**Support Resolution**")
    print(final_result)
    
finally:
    await runtime.stop_when_idle()

🎯 Customer Support Scenario: Technical Issue
**SupportTriage**: 
  📞 Calling: Handoff-transfer_to_TechSupport({})
--------------------------------------------------
**SupportTriage**: 
  📋 Result: None
--------------------------------------------------
**SupportTriage**: 
--------------------------------------------------
**TechSupport**: 
  📞 Calling: TechnicalSupportPlugin-diagnose_issue({"issue":"User's device keeps disconnecting from the internet."})
--------------------------------------------------
**TechSupport**: 
  📋 Result: Diagnosis for 'User's device keeps disconnecting from the internet.': This appears to be a common connectivity issue. Try restarting your device and checking your network connection.
--------------------------------------------------
**TechSupport**: 
  📞 Calling: TechnicalSupportPlugin-troubleshoot({"diagnosis":"Common connectivity issue. Try restarting your device and checking your network connection."})
--------------------------------------------------

## 6. Human-in-the-Loop Systems

Sometimes you need human input during agent interactions. Here's how to implement human-in-the-loop systems.

In [27]:
# Custom manager that requests human input
class HumanInTheLoopManager(RoundRobinGroupChatManager):
    """Manager that requests human input after specific conditions."""
    
    @override
    async def should_request_user_input(self, chat_history: ChatHistory) -> BooleanResult:
        """Request user input after every 2 agent exchanges."""
        agent_messages = [msg for msg in chat_history.messages if msg.role == AuthorRole.ASSISTANT]
        
        if len(agent_messages) >= 2 and len(agent_messages) % 2 == 0:
            return BooleanResult(
                result=True,
                reason="Requesting user feedback after agent exchanges"
            )
        
        return BooleanResult(
            result=False,
            reason="No user input needed yet"
        )

# Simple human response function for demonstration
def get_human_response(chat_history: ChatHistory) -> ChatMessageContent:
    """Get human input (simulated for this example)."""
    # In a real application, this would get actual user input
    responses = [
        "That's interesting! Can you elaborate on the benefits?",
        "I like the first approach better. Can you refine it?",
        "Good work! Please finalize the solution."
    ]
    
    # Simulate human response based on conversation length
    response_index = min(len(chat_history.messages) // 4, len(responses) - 1)
    human_input = responses[response_index]
    
    print(f"👤 Human: {human_input}")
    return ChatMessageContent(role=AuthorRole.USER, content=human_input)

print("✅ Human-in-the-loop components created!")

✅ Human-in-the-loop components created!


In [28]:
# Create collaborative agents
def create_collaborative_agents() -> list[Agent]:
    """Create agents that work well with human feedback."""
    
    strategist = ChatCompletionAgent(
        name="Strategist",
        description="A strategic planner.",
        instructions="You develop high-level strategies and adapt based on feedback. Be responsive to human input.",
        service=chat_completion,
    )
    
    implementer = ChatCompletionAgent(
        name="Implementer",
        description="A solution implementer.",
        instructions="You create detailed implementation plans based on strategies. Incorporate human feedback.",
        service=chat_completion,
    )
    
    return [strategist, implementer]

# Create human-in-the-loop orchestration
collaborative_agents = create_collaborative_agents()
human_loop_orchestration = GroupChatOrchestration(
    members=collaborative_agents,
    manager=HumanInTheLoopManager(
        max_rounds=6,
        human_response_function=get_human_response,
    ),
    agent_response_callback=agent_response_callback,
)

print("✅ Human-in-the-loop orchestration created!")

✅ Human-in-the-loop orchestration created!


In [29]:
# Test human-in-the-loop system
runtime = InProcessRuntime()
runtime.start()

print("🎯 Collaborative Project: Remote Team Productivity")
print("=" * 60)

try:
    orchestration_result = await human_loop_orchestration.invoke(
        task="Develop a strategy to improve remote team productivity and collaboration.",
        runtime=runtime,
    )
    
    final_result = await orchestration_result.get()
    print(f"\n**Final Collaborative Solution**")
    print(final_result)
    
finally:
    await runtime.stop_when_idle()

🎯 Collaborative Project: Remote Team Productivity
**Strategist**
Here’s a high-level, phased strategy for boosting productivity and collaboration across your remote team. You can tailor each step to your organization’s size, culture and tooling.

Phase 1 – Discovery & Alignment  
1. Audit current state  
   • Survey team on pain points, tool-usage and communication gaps  
   • Review existing workflows, meeting cadences and project metrics  
2. Define clear objectives  
   • Establish 2–4 key productivity and collaboration goals (e.g., reduce task hand-off ti... [truncated]
--------------------------------------------------
**Implementer**
Implementation Plan: Improving Remote Team Productivity & Collaboration

Below is a 12-week rollout plan with clear tasks, owners, checkpoints and feedback loops. You can tailor timing or team size as needed.

--------------------------------  
Phase 1 (Weeks 1–2): Discovery & Alignment  
Objective: Build shared situational awareness and define measu

## 🎉 Congratulations!

You've successfully completed the Multi-Agent Orchestration tutorial! Here's what you've mastered:

### ✅ Orchestration Patterns:
1. **Group Chat Orchestration** - Round-robin managers for structured conversations
2. **Concurrent Processing** - Parallel agent execution with structured outputs
3. **Sequential Workflows** - Pipeline-based processing with streaming capabilities
4. **LLM-Based Group Chat Manager** - AI-powered intelligent speaker selection
5. **Handoff Systems** - Specialized agent routing for complex workflows
6. **Human-in-the-Loop** - Interactive agent workflows with user input

### 🔧 Key Concepts:
- **Runtime Management** - Using `InProcessRuntime` for agent execution
- **Callback Functions** - Monitoring agent responses and function calls
- **Streaming Integration** - Real-time updates in orchestration workflows
- **Structured Outputs** - Consistent results across multiple agents
- **Agent Specialization** - Creating focused agents with specific roles
- **Intelligent Mediation** - Using LLM to make orchestration decisions

### 🤖 Advanced Features:
- **AI-Powered Selection** - LLM-based managers that understand conversation context
- **Reasoning and Transparency** - Managers that explain their decisions
- **Adaptive Termination** - Smart detection of when discussions should end
- **Context-Aware Routing** - Intelligent handoffs based on conversation content

### 📚 Recommended Next step:
- Review [Semantic Kernel Samples](https://github.com/microsoft/semantic-kernel/tree/main/python/samples)

### 🤝 Community
- Check out the [Developer Blog](https://devblogs.microsoft.com/semantic-kernel/)

Excellent work! You're now ready to start building multi-agent systems. 🚀🤖